In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%cd /kaggle/working
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

In [ ]:
%cd chatGLM-6B-QLoRA/
!git pull --all --force 
!pip install -r requirements.txt

In [ ]:
ls

# 测试  新的 train_lora.py

In [ ]:
!git pull --all --force
!deepspeed --include localhost:1  train_lora.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b  \
  --output_dir output0908_v2 \
  --resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/output0908/checkpoint-200 \
  --num_eval_samples 1 \
  --train_data_path data/alpaca_yunguan_0814 \
  --eval_data_path  data/alpaca_yunguan_0814 \
  --max_input_length 256 \
  --max_output_length 256 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 4 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

# 测试  新的 sft_lora_multi_turn.py

In [ ]:
!git pull --all --force
!deepspeed --include localhost:0,1  sft_lora_multi_turn.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-sharegpt-2k-sft-0908 \
  --num_train_samples -1 \
  --num_eval_samples 2 \
  --train_data_path ./data/sharegpt_data  \
  --eval_data_path  ./data/sharegpt_data    \
  --data_type sharegpt  \
  --max_length 1000 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 2 \
  --learning_rate  1.8e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

# 构造和测试langchain prompt 数据 已经将构造的数据上传到github项目

In [ ]:
import json 
import os

template = """现提供如下信息：
history={history}
context={context}
请使用上述信息回答：{question}
"""

parent_dir = "/kaggle/working/chatGLM-6B-QLoRA/data/alpaca_yunguan_0814"
paths = ['yunguan_aug_part_3.json',
         'yunguan_aug_part_2.json',
        "yunguan_aug_part_1.json",
        "refuse_phone.json"]
list_all =[]
for name in paths :
    with open(os.path.join(parent_dir , name) ,"r",encoding="utf-8") as fr:
        list_all.extend([json.loads(item) for item in  fr.readlines() ])

print(len(list_all))


def get_q_a(idx):
    q = list_all[idx]["instruction"]
    a = list_all[idx]["output"]
    return "\n\n问：{q}\n\n答：{a}".format(q=q,a=a)

import random
res = []
idx_list_all = range(len(list_all))
for chosen_i in range(len(list_all)) :
    his_idx = random.choices(idx_list_all,k=2)
    #print(his_idx)
    context_idx = random.choices(idx_list_all,k=2) 
    #print(context_idx)
    context_idx.extend([chosen_i]) # 两个不相关检索答案+ 正确检索答案 ,比如 [1,2,100] 其中100是正确的q和a 等会idx=100的q作为question
    random.shuffle(context_idx) # 打乱上下文的顺序 [1,100,2]
    history = "".join([get_q_a(idx) for idx in his_idx])
    context = "".join([get_q_a(idx) for idx in context_idx])
    question = list_all[chosen_i]["instruction"]
    answer = list_all[chosen_i]["output"]
    res.append({"instruction":template.format(history=history,context=context,question=question)  ,"input":"" , "output":answer })

# print(res[-10]['instruction'])
# print(res[-10]['output'])

max_q ,max_a, max_total = 0,0,0

for item in res:
    max_q = max(max_q, len(item["instruction"]))
    max_a = max(max_a, len(item['output']))
    max_total = max(max_total , len(item["instruction"]) + len(item["output"]) )

with open(f"alpaca_langchain_prompt_yunguan_0922_max_q_{max_q}_max_a_{max_a}_max_total_{max_total}.json","w",encoding="utf-8") as fw:
    for item in res :
        fw.write(json.dumps(item,ensure_ascii=False)+"\n")
print(f"alpaca_langchain_prompt_yunguan_0922_max_q_{max_q}_max_a_{max_a}_max_total_{max_total}.json   done")

In [ ]:

#cat alpaca_langchain_prompt_yunguan_0919.json

# 用alpaca格式的langchain prompt新数据跑

In [ ]:
!git pull --all --force

In [ ]:
!git pull --all --force
!deepspeed --include localhost:0,1  train_lora.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b  \
  --output_dir output_langchain_yunguan_0922 \
  --num_eval_samples 100 \
  --train_data_path data/alpaca_langchain_yunguan \
  --eval_data_path  data/alpaca_langchain_yunguan \
  --max_input_length 900 \
  --max_output_length 256 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 4 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json

In [ ]:
!git pull --all --force
!deepspeed --include localhost:0,1  train_lora.py \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b  \
  --output_dir output_langchain_yunguan_0922_v2 \
  --resume_from_checkpoint /kaggle/working/chatGLM-6B-QLoRA/output_langchain_yunguan_0922/checkpoint-200 \
  --num_eval_samples 100 \
  --train_data_path data/alpaca_langchain_yunguan \
  --eval_data_path  data/alpaca_langchain_yunguan \
  --max_input_length 900 \
  --max_output_length 256 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 4 \
  --learning_rate  1e-6 \
  --num_train_epochs  70  \
  --save_total_limit 2 \
  --use_qlora True \
--deepspeed ds_zero2_config.json